# Fine Tuning Roberta for Sentiment Analysis





Note: Most of this was reused from [this Collab notebook](https://colab.research.google.com/github/DhavalTaunk08/NLP_scripts/blob/master/sentiment_analysis_using_roberta.ipynb
), which demostrates fine-tuning roberta on sentiment analysis (from huggingface resources docs on roberta)

### Introduction

We're going to use a roberta model for **spam vs ham classification**. "spam" labels define spam (self-explanatory), and "ham" labels define a message sent by a homie (probably).

### Table of Contents
1. [Imports](#section01)
2. [Data Pre-processing](#section02)
3. [Data Prep](#section03)
4. [Creating the Neural Net](#section04)
5. [Fine Tuning the Model](#section05)
6. [Testing the Model](#section06)
7. [Saving the Model](#section07)


#### Technical Details


- **Data**: I used 3 datasets: two for fine-tuning, one for classification.
	- [Enron](https://huggingface.co/datasets/SetFit/enron_spam/blob/main/enron_spam_data.csv) (email data, but using only the messages) - this is part of the training data.
	- [Telegram](https://huggingface.co/datasets/thehamkercat/telegram-spam-ham/blob/main/dataset.csv) - this is part of the training data.
	- [Spam SMS Classification Using NLP](https://www.kaggle.com/code/shadymohammed205/nlp-meets-sms-distilbert-for-spam-detection) dataset from the Kaggle Competition - this is our test dataset.


- **Language Model**: I used RoBERTa, as seen in [this research paper](https://arxiv.org/pdf/1907.11692). Its architecture was based on Google’s encoder-only BERT model released back in 2018 (which at the time was significantly undertrained, according to the authors of the paper), which is now a staple among NLP-ists.
	- [Blog-Post](https://ai.facebook.com/blog/roberta-an-optimized-method-for-pretraining-self-supervised-nlp-systems/)
	- [Documentation for python](https://huggingface.co/transformers/model_doc/roberta.html)

- **Hardware Requirements**: I ran this on Google Collab using a A100 GPU (when in Rome...).
	- Strongly recommend this to be a GPU enabled setup, it's a 5x fine-tuning difference between a T4 and an A100, and I can imagine the CPU to be even slower.
	- Since this is Google Collab, we're using Python 3.10+, but I think most of these require 3.6+

In [ ]:
!python --version

Python 3.10.12


<a id='section01'></a>
### Imports

In [ ]:
# Need to install transformers in Google Collab
!pip install transformers

In [ ]:
# Importing the necessaries
import pandas as pd
import numpy as np

import json

import seaborn as sns

from sklearn.model_selection import train_test_split

import torch
import transformers
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaModel, RobertaTokenizer

import logging
logging.basicConfig(level=logging.ERROR)

In [ ]:
# If you got that sweet CUD...
# As cheese? You gouda.
# On CPU you must sort?
# You smell like Rockfort.
# Just kidding, this is all part of me losing my sweet mind :)

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

<a id='section02'></a>
### Data Pre-processing

We use Enron + Twitter as our training data, and our test data is the original kaggle competition

In [ ]:
# Enron - remember, since these collab instances are ephemeral, you need to upload
enron_df = pd.read_csv('data/enron_spam_data.csv')
enron_df = enron_df[['Spam/Ham', 'Message']].dropna().drop_duplicates('Message')
enron_df.columns = ['label', 'message']

# Twitter
telegram_df = pd.read_csv('data/telegram_spam_ham_data.csv').dropna().drop_duplicates('text')
telegram_df.columns = ['label', 'message']

In [ ]:
train_df = pd.concat((enron_df, telegram_df), ignore_index=True)

In [ ]:
test_df = pd.read_csv('data/Spam_SMS.csv')
test_df.columns = test_df.columns.map(str.lower)
test_df.columns = ['label', 'message']

In [ ]:
print(train_df.message.apply(len).max())
print(test_df.message.apply(len).max())

228353
910


In [ ]:
train_df['label'] = train_df['label'].apply(lambda x: 1 if x=='spam' else 0)
test_df['label'] = test_df['label'].apply(lambda x: 1 if x=='spam' else 0)


<a id='section03'></a>
### Data Prep
Most of this is kept the same from the original author's work.
Some differences are renaming of the Dataset class, and changing the number of classes from 5 (sentiments) to 2 (spam/!spam). This is done in the Dataset class, the "pre-processor" to the NN.

We use the [Roberta tokenizer](https://huggingface.co/transformers/model_doc/roberta.html#robertatokenizer), with encode_plus method to generate `ids` and `attention_mask` as outputs; `target` is the spam / not spam label.

Originally, the author had used a training / testing split, but in our case we are using two explicitly differently distributed and not overlapping predefined datasets. We still use a validation function though.

The Dataloader is used to for creating training and validation dataloader that loads data to the neural network in a defined manner. The original author of this collab had to split it since VRAM was limited to (I think) 12GB on the T4, but using the A100, it's increased to 40, which helps. Regardless, most of these are kept the same.Original author links further reading for Dataset and Dataloader [here](https://pytorch.org/docs/stable/data.html).

#### **SpamOrHamData** Dataset Class

- The tokenizer uses the `encode_plus` method to perform tokenization and generate the necessary outputs,method to perform tokenization and generate the necessary outputs, namely: `ids`, `attention_mask`
- To read further into the tokenizer, [refer to this document](https://huggingface.co/transformers/model_doc/roberta.html#robertatokenizer)
- `target` is the encoded category on the news headline.
- The *SentimentData* class is used to create 2 datasets, for training (enron + telegram) and for testing (telegram)

#### Dataloader


In [ ]:
# Defining some key variables that will be used later on in the training
MAX_LEN = 512
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 4
LEARNING_RATE = 1e-05
tokenizer = RobertaTokenizer.from_pretrained('roberta-base', truncation=True, do_lower_case=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
class SpamOrHamData(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.message
        self.targets = self.data.label
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [ ]:
training_set = SpamOrHamData(train_df, tokenizer, MAX_LEN)
testing_set = SpamOrHamData(test_df, tokenizer, MAX_LEN)

In [ ]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

<a id='section04'></a>
### Creating the Neural Net
(From Original Author, this part is largely unchanged)
#### Neural Network
 - We will be creating a neural network with the `RobertaClass`.
 - This network will have the Roberta Language model followed by a `dropout` and finally a `Linear` layer to obtain the final outputs.
 - The data will be fed to the Roberta Language model as defined in the dataset.
 - Final layer outputs is what will be compared to the `Sentiment category` to determine the accuracy of models prediction.
 - We will initiate an instance of the network called `model`. This instance will be used for training and then to save the final trained model for future inference.

#### Loss Function and Optimizer
 - `Loss Function` and `Optimizer` and defined in the next cell.
 - The `Loss Function` is used the calculate the difference in the output created by the model and the actual output.
 - `Optimizer` is used to update the weights of the neural network to improve its performance.

In [ ]:
class RobertaClass(torch.nn.Module):
    def __init__(self):
        super(RobertaClass, self).__init__()
        self.l1 = RobertaModel.from_pretrained("roberta-base")
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(768, 2)

    def forward(self, input_ids, attention_mask, token_type_ids):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

In [ ]:
model = RobertaClass()
model.to(device)

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaClass(
  (l1): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((

<a id='section05'></a>
### Fine Tuning the Model

In [ ]:
# Creating the loss function and optimizer
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [ ]:
def calcuate_accuracy(preds, targets):
    n_correct = (preds==targets).sum().item()
    return n_correct

In [ ]:
# Defining the training function on the 80% of the dataset for tuning the distilbert model

def train(epoch):
    tr_loss = 0
    n_correct = 0
    nb_tr_steps = 0
    nb_tr_examples = 0
    model.train()
    for _,data in tqdm(enumerate(training_loader, 0)):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.long)

        outputs = model(ids, mask, token_type_ids)
        loss = loss_function(outputs, targets)
        tr_loss += loss.item()
        big_val, big_idx = torch.max(outputs.data, dim=1)
        n_correct += calcuate_accuracy(big_idx, targets)

        nb_tr_steps += 1
        nb_tr_examples+=targets.size(0)

        if _%5000==0:
            loss_step = tr_loss/nb_tr_steps
            accu_step = (n_correct*100)/nb_tr_examples
            print(f"Training Loss per 5000 steps: {loss_step}")
            print(f"Training Accuracy per 5000 steps: {accu_step}")

        optimizer.zero_grad()
        loss.backward()
        # # When using GPU
        optimizer.step()

    print(f'The Total Accuracy for Epoch {epoch}: {(n_correct*100)/nb_tr_examples}')
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Training Loss Epoch: {epoch_loss}")
    print(f"Training Accuracy Epoch: {epoch_accu}")

    return

In [ ]:
EPOCHS = 3
for epoch in range(EPOCHS):
    train(epoch)

0it [00:00, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2870: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Training Loss per 5000 steps: 0.6652650237083435
Training Accuracy per 5000 steps: 37.5


5001it [15:35,  5.35it/s]

Training Loss per 5000 steps: 0.09503122924742213
Training Accuracy per 5000 steps: 96.53819236152769


6265it [19:31,  5.35it/s]


The Total Accuracy for Epoch 0: 96.82916608464869
Training Loss Epoch: 0.08699450095377213
Training Accuracy Epoch: 96.82916608464869


0it [00:00, ?it/s]

Training Loss per 5000 steps: 0.0015362361446022987
Training Accuracy per 5000 steps: 100.0


1152it [03:35,  5.36it/s]


KeyboardInterrupt: 

<a id='section06'></a>
### Testing the Model

Our data split is about 90%-10%, with different distributions. I think by mixing the three, we can definitely increase performance from ~95% upwards.

In [ ]:
def valid(model, testing_loader):
    model.eval()
    n_correct = 0; n_wrong = 0; total = 0; tr_loss=0; nb_tr_steps=0; nb_tr_examples=0
    with torch.no_grad():
        for _, data in tqdm(enumerate(testing_loader, 0)):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
            targets = data['targets'].to(device, dtype = torch.long)
            outputs = model(ids, mask, token_type_ids).squeeze()
            loss = loss_function(outputs, targets)
            tr_loss += loss.item()
            big_val, big_idx = torch.max(outputs.data, dim=1)
            n_correct += calcuate_accuracy(big_idx, targets)

            nb_tr_steps += 1
            nb_tr_examples+=targets.size(0)

            if _%5000==0:
                loss_step = tr_loss/nb_tr_steps
                accu_step = (n_correct*100)/nb_tr_examples
                print(f"Validation Loss per 100 steps: {loss_step}")
                print(f"Validation Accuracy per 100 steps: {accu_step}")
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Validation Loss Epoch: {epoch_loss}")
    print(f"Validation Accuracy Epoch: {epoch_accu}")

    return epoch_accu


In [ ]:
acc = valid(model, testing_loader)
print("Accuracy on test data = %0.2f%%" % acc)

6it [00:00, 26.19it/s]

Validation Loss per 100 steps: 0.0016986319096758962
Validation Accuracy per 100 steps: 100.0


1394it [00:48, 28.87it/s]

Validation Loss Epoch: 0.12748833583852476
Validation Accuracy Epoch: 95.03049874416936
Accuracy on test data = 95.03%


<a id='section07'></a>
### Saving The Model



In [ ]:
output_model_file = 'pytorch_roberta_spam.bin'
output_vocab_file = './'

model_to_save = model
torch.save(model_to_save, output_model_file)
tokenizer.save_vocabulary(output_vocab_file)

print('All files saved')
print('This tutorial is completed')

All files saved
This tutorial is completed
